In [4]:
!pip install ttkbootstrap

import ttkbootstrap as ttk
from ttkbootstrap.constants import *
from tkinter import messagebox
import pandas as pd
import random

# Extend movie dataset
def extend_movie_dataset(csv_path):
    df = pd.read_csv(csv_path)
    new_movies = [
        {"Title": "Interstellar", "Genre": "Adventure, Sci-Fi", "Director": "Christopher Nolan", "Actors": "Matthew McConaughey, Anne Hathaway", "Year": 2014, "Rating": 8.6, "Votes": 1500000, "Description": "Explorers travel through a wormhole to ensure humanity's survival."},
        {"Title": "The Dark Knight", "Genre": "Action, Crime, Drama", "Director": "Christopher Nolan", "Actors": "Christian Bale, Heath Ledger", "Year": 2008, "Rating": 9.0, "Votes": 2300000, "Description": "Batman faces the Joker, a criminal mastermind who plunges Gotham into chaos."}
    ]
    for movie in new_movies:
        if not ((df['Title'] == movie['Title']) & (df['Year'] == movie['Year'])).any():
            df = pd.concat([df, pd.DataFrame([movie])], ignore_index=True)
    df.to_csv(csv_path, index=False)
    return df

# Mood logic
questions = [
    {"question": "Pick your ideal evening activity", "options": ["Solving puzzles", "Going to a concert", "Watching stand-up comedy", "Reading sci-fi novels"]},
    {"question": "Choose a color that speaks to your mood", "options": ["Blue", "Red", "Yellow", "Black"]},
    {"question": "How are you feeling today?", "options": ["Excited", "Curious", "Chill", "Adventurous"]}
]

mood_map = {
    "Solving puzzles": "Thriller", "Going to a concert": "Drama", "Watching stand-up comedy": "Comedy", "Reading sci-fi novels": "Sci-Fi",
    "Blue": "Drama", "Red": "Action", "Yellow": "Comedy", "Black": "Thriller",
    "Excited": "Action", "Curious": "Sci-Fi", "Chill": "Drama", "Adventurous": "Adventure"
}

genre_map = {
    "Thriller": ["Thriller", "Crime"], "Drama": ["Drama"], "Comedy": ["Comedy"],
    "Sci-Fi": ["Sci-Fi", "Adventure"], "Action": ["Action"], "Adventure": ["Adventure", "Sci-Fi"]
}

def predict_mood(answers):
    mood_count = {}
    for ans in answers:
        mood = mood_map.get(ans, "Drama")
        mood_count[mood] = mood_count.get(mood, 0) + 1
    return max(mood_count, key=mood_count.get)

def recommend_movies(df, mood):
    genres = genre_map.get(mood, ["Drama"])
    filtered = df[df["Genre"].str.contains('|'.join(genres), case=False, na=False)]
    return filtered.sort_values(["Rating", "Votes"], ascending=[False, False]).head(3)

def filter_by_director(df, name):
    return df[df["Director"].str.contains(name, case=False, na=False)]

def filter_by_cast(df, name):
    return df[df["Actors"].str.contains(name, case=False, na=False)]

def surprise_movie(df):
    top = df.sort_values(["Rating", "Votes"], ascending=[False, False])
    return top.sample(1)

class MovieRecommenderApp:
    def __init__(self, root, df):
        self.root = root
        self.df = df
        self.answers = []
        self.q_index = 0
        self.var = None

        self.root.title("🎬 Mood-Based Movie Recommender")
        self.root.geometry("700x600")
        self.frame = ttk.Frame(root, padding=20, bootstyle='dark')
        self.frame.pack(fill="both", expand=True)

        self.show_question()

    def clear_frame(self):
        for widget in self.frame.winfo_children():
            widget.destroy()

    def show_question(self):
        self.clear_frame()
        if self.q_index < len(questions):
            q = questions[self.q_index]
            ttk.Label(self.frame, text=f"Q{self.q_index + 1}: {q['question']}", font=("Segoe UI", 16, "bold")).pack(pady=15)

            self.var = ttk.StringVar(value="__none__")  # Ensures no selection by default

            for opt in q["options"]:
                ttk.Radiobutton(self.frame, text=opt, variable=self.var, value=opt,
                                command=self.on_option_selected, bootstyle="info").pack(anchor="w", pady=5)

            self.next_button = ttk.Button(self.frame, text="Next", command=self.next_question, bootstyle="success", state=DISABLED)
            self.next_button.pack(pady=20)
        else:
            self.ask_for_filters()

    def on_option_selected(self):
        if self.var.get() != "__none__":
            self.next_button.config(state=NORMAL)

    def next_question(self):
        ans = self.var.get()
        if ans == "__none__":
            messagebox.showwarning("⚠️", "Please select an option!")
            return
        self.answers.append(ans)
        self.q_index += 1
        self.show_question()

    def ask_for_filters(self):
        self.clear_frame()
        ttk.Label(self.frame, text="🎯 Optional Filters", font=("Segoe UI", 16, "bold")).pack(pady=15)

        ttk.Label(self.frame, text="🎬 Filter by Director").pack(anchor="w")
        self.director_entry = ttk.Entry(self.frame, width=40)
        self.director_entry.pack(anchor="w", pady=5)

        ttk.Label(self.frame, text="🎭 Filter by Cast").pack(anchor="w")
        self.cast_entry = ttk.Entry(self.frame, width=40)
        self.cast_entry.pack(anchor="w", pady=5)

        ttk.Button(self.frame, text="🎥 Show Recommendations", command=self.show_recommendations, bootstyle="primary").pack(pady=10)
        ttk.Button(self.frame, text="🎲 Surprise Me", command=self.show_surprise, bootstyle="warning-outline").pack()

    def show_recommendations(self):
        mood = predict_mood(self.answers)
        df = self.df.copy()

        director_name = self.director_entry.get().strip()
        cast_name = self.cast_entry.get().strip()
        if director_name:
            df = filter_by_director(df, director_name)
        if cast_name:
            df = filter_by_cast(df, cast_name)

        if df.empty:
            messagebox.showinfo("No Match", "No movies found with your filter. Try again!")
            return

        recs = recommend_movies(df, mood)
        self.clear_frame()

        ttk.Label(self.frame, text=f"🧠 Your Mood: {mood}", font=("Segoe UI", 15, "bold"), bootstyle="info").pack(pady=10)
        ttk.Label(self.frame, text="🎥 Top 3 Recommendations:", font=("Segoe UI", 13, "bold")).pack(pady=5)

        for _, row in recs.iterrows():
            card = ttk.Frame(self.frame, padding=10, bootstyle="light",relief="",borderwidth=2)
            card.pack(fill="x", pady=8)

            ttk.Label(card, text=f"🎬 {row['Title']} ({row['Genre']})", font=("Segoe UI", 12, "bold")).pack(anchor="w")
            ttk.Label(card, text=f"⭐ Rating: {row['Rating']}   🎬 Director: {row['Director']}", font=("Calibri", 10)).pack(anchor="w", pady=2)
            ttk.Label(card, text=f"🎭 Cast: {row['Actors']}", font=("Calibri", 10)).pack(anchor="w", pady=2)
            ttk.Label(card, text=row["Description"], font=("Calibri", 9), wraplength=650, justify="left").pack(anchor="w", pady=2)

        ttk.Button(self.frame, text="🔁 Restart", command=self.restart, bootstyle="secondary").pack(pady=15)

    def show_surprise(self):
        surprise = surprise_movie(self.df).iloc[0]
        msg = f"{surprise['Title']} ({surprise['Genre']})\n\n{surprise['Description']}\n\nRating: {surprise['Rating']}"
        messagebox.showinfo("🎬 Surprise Movie!", msg)

    def restart(self):
        self.answers = []
        self.q_index = 0
        self.show_question()

if __name__ == "__main__":
    csv_path = "IMDB-Movie-Data.csv"
    df = extend_movie_dataset(csv_path)
    root = ttk.Window(title="🎬 Mood-Based Movie Recommender", themename="darkly")
  # Try themes like "darkly", "cyborg", etc.
    app = MovieRecommenderApp(root, df)
    root.mainloop()